Задача 1. Выбор кредита от Tinkoff.ru
В credit_train.csv содержится 170746 строк с данными о клиентах сети магазинов электроники, в этих магазинах они подали заявки на кредит. Колонка open_account_flg содержит 1 если клиент выбрал Тинькофф и 0 в противном случае. В credit_test.csv содержится 91940 строк с данными, для каждой строки следует предсказать возьмет ли соответствующий ей человек кредит в Тинькофф.
Метрикой качества в задаче является AUC 

education - образование:

    SCH - начальное, среднее
    PGR -  второе высшее
    GRD - высшее
    UGR - неполное высшее
    ACD - ученая степень
    
    
job_position - работа

    SPC - неруководящий сотрудник, специалист
    DIR - руководитель организации
    HSK - домохозяйка
    INV - не работает, инвалидность
    WOI - работает на ИП
    WRK - неруководящий сотрудник, рабочий
    UMN -  руководитель подразделения
    NOR - не работает
    PNS - пенсионер
    BIS - собственный бизнес 
    INP - ИП
   
marital_status - семейное положение

    UNM - холост/не замужем
    DIV - разведен(а)
    MAR - женат/замужем
    WID - вдовец/вдова
    CIV - гражданский брак
    
gender - пол 

age - возраст

credit_sum - сумма кредита

credit_month - срок кредитования

tariff_id - номер предлагаемого тарифа

living_region - регион проживания

montly_income - заработная плата в месяц

credit_count - количество кредитов у клиента

overdue_credit_count - количество просроченных кредитов

score_shk - неизвестный индекс кредитоспособности
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation

In [ ]:
train = pd.read_csv("../credit_train.csv", sep = ";", encoding='cp1251')
test = pd.read_csv("../credit_test.csv", sep = ";", encoding='cp1251')

In [ ]:
train.head(2)

In [ ]:
y = train['open_account_flg']

In [ ]:
train.info()

In [ ]:
test.info()

I работа с пропусками в данных

In [ ]:
train[train.monthly_income.isnull() == True]

In [ ]:
train.monthly_income.fillna(train.monthly_income.mean(), inplace=True)

In [ ]:
train.living_region.fillna(value="Unknown", inplace=True)
test.living_region.fillna(value="Unknown", inplace=True)

In [ ]:
train.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
train.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [ ]:
test.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
test.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [ ]:
train.job_position.unique()

In [ ]:
train.marital_status.unique()

In [ ]:
train.groupby("job_position").target.mean()

In [ ]:
train.groupby("education").target.mean()

In [ ]:
train.groupby("marital_status").target.mean()

In [ ]:
train.groupby("tariff_id").target.mean()

In [ ]:
test[test.job_position == "INV"]

In [ ]:
train.gender = train.gender.map({"M":1, "F":0})
test.gender = test.gender.map({"M":1, "F":0})

train.rename(columns={"open_account_flg":"target"}, inplace=True)

II сделать сбалансированные классы

In [ ]:
count_to_add = abs (np.sum(train.target == 1) - np.sum(train.target == 0))
indices_to_add = np.random.randint(np.sum(train.target == 1), size = count_to_add)
train_to_add = pd.DataFrame(data = train.values[train.target.as_matrix() == 1][indices_to_add,:],
                            columns=train.columns)
train = pd.concat([train, train_to_add], axis=0)
y = train.target

II выделим группы людей по наличию высшего образования, работе и пенсии 

In [ ]:
### по образованию
train["high_edu"] = ((train["education"] != "SCH") & (train["education"] != "UGR"))

### по работе
# 1. не работающие граждание
train["not_work"] = ((train.job_position == "HSK") | (train.job_position == "INV") 
                     | (train.job_position == "NOR") | (train.job_position == "PNS"))

# 2. обычные работники
train["usual_workers"] = ((train.job_position == "SPC") | (train.job_position == "WOI") 
                     | (train.job_position == "WRK") | (train.job_position == "ATP") 
                     | (train.job_position == "WRP"))

# 3. начальники
train["head_workers"] = ((train.job_position == "DIR") | (train.job_position == "UMN")
                         | (train.job_position == "BIS") | (train.job_position == "INP"))

# 4. имеют собственный бизнес
train["have_business"] = ((train.job_position == "BIS") | (train.job_position == "INP"))

### по браку
train["single"] = ((train.marital_status == "UNM") | (train.marital_status == "DIV")
                   | (train.marital_status == "WID"))

In [ ]:
### по образованию
test["high_edu"] = ((test["education"] != "SCH") & (test["education"] != "UGR"))

### по работе
# 1. не работающие граждание
test["not_work"] = ((test.job_position == "HSK") | (test.job_position == "INV") 
                     | (test.job_position == "NOR") | (test.job_position == "PNS"))

# 2. обычные работники
test["usual_workers"] = ((test.job_position == "SPC") | (test.job_position == "WOI") 
                     | (test.job_position == "WRK") | (test.job_position == "ATP") 
                     | (test.job_position == "WRP"))

# 3. начальники
test["head_workers"] = ((test.job_position == "DIR") | (test.job_position == "UMN")
                         | (test.job_position == "BIS") | (test.job_position == "INP"))

# 4. имеют собственный бизнес
test["have_business"] = ((test.job_position == "BIS") | (test.job_position == "INP"))

### по браку
test["single"] = ((test.marital_status == "UNM") | (test.marital_status == "DIV")
                   | (test.marital_status == "WID"))

III кодировка категориальных признаков, кроме living region

In [ ]:
train_col = train.columns.drop("target")
dummy_df = pd.concat([train[train_col], test], axis=0)
dummy_df = pd.get_dummies(dummy_df, columns=["education","job_position", "marital_status"], drop_first=True)
train = dummy_df[:train.shape[0]]
test = dummy_df[train.shape[0]:]

In [ ]:
train.columns

IV кодировка региона через среднюю зарплату

In [ ]:
def code_mean(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

In [ ]:
train['monthly_income'] = train['monthly_income'].astype(float)
test['monthly_income'] = test['monthly_income'].astype(float)

train["city_mean_income"] = code_mean(train, 'living_region', 'monthly_income')
train.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
train.drop("living_region", axis=1, inplace=True)

test["city_mean_income"] = code_mean(test, 'living_region', 'monthly_income')
test.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
test.drop("living_region", axis=1, inplace=True)

V Генерация данных о зарплате и выплатам по кредиту

In [ ]:
train['credit_sum'] = train['credit_sum'].apply(lambda x: x.replace(",","."))
test['credit_sum'] = test['credit_sum'].apply(lambda x: x.replace(",","."))

train['credit_sum'] = train['credit_sum'].astype(float)
test['credit_sum'] = test['credit_sum'].astype(float)

credit_pay - ежемесячный расход на кредит

money_for_life - разность между зп и платой по кредиту

dif_city - разница между зп и средней зп по региону

In [ ]:
train["credit_pay"] = train["credit_sum"] / train["credit_month"]
train["money_for_life"] = train["monthly_income"] - train["credit_pay"]
train["dif_city"] = train["monthly_income"] - train["city_mean_income"]

test["credit_pay"] = test["credit_sum"] / test["credit_month"]
test["money_for_life"] = test["monthly_income"] - test["credit_pay"]
test["dif_city"] = test["monthly_income"] - test["city_mean_income"]

VI анализ score_shk

In [ ]:
train['score_shk'] = train['score_shk'].apply(lambda x: x.replace(",","."))
test['score_shk'] = test['score_shk'].apply(lambda x: x.replace(",","."))

train['score_shk'] = train['score_shk'].astype(float)
test['score_shk'] = test['score_shk'].astype(float)

In [ ]:
train['target'] = y

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.to_csv("train_random_oversampling.csv", columns=train.columns, index=False)
test.to_csv("test.csv", columns=test.columns, index=False)